<a href="https://colab.research.google.com/github/rishabkolan/Predicting-the-amount-of-time-a-UFC-fight-lasts/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **UFC fight duration prediction**

# 1. Data Preprocessing

In [119]:
#Importing the Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

In [120]:
#Creating the dataframe

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Classroom/large_dataset.csv'
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [121]:
df.head(5)

,event_name,r_fighter,b_fighter,winner,weight_class,is_title_bout,gender,method,finish_round,total_rounds,...,weight_diff,reach_diff,SLpM_total_diff,SApM_total_diff,sig_str_acc_total_diff,td_acc_total_diff,str_def_total_diff,td_def_total_diff,sub_avg_diff,td_avg_diff
0,UFC Fight Night: Ribas vs. Namajunas,Amanda Ribas,Rose Namajunas,Blue,Women's Flyweight,0,Women,Decision - Unanimous,5,5.0,...,0.00,2.54,0.94,-0.11,-0.01,0.04,-0.02,0.26,0.2,0.69
1,UFC Fight Night: Ribas vs. Namajunas,Karl Williams,Justin Tafa,Red,Heavyweight,0,Men,Decision - Unanimous,3,3.0,...,-13.16,12.70,-1.22,-3.32,-0.02,0.50,0.13,0.50,0.2,4.75
2,UFC Fight Night: Ribas vs. Namajunas,Edmen Shahbazyan,AJ Dobson,Red,Middleweight,0,Men,KO/TKO,1,3.0,...,0.00,-2.54,-0.69,-1.22,0.06,-0.37,-0.01,-0.02,0.3,0.57
3,UFC Fight Night: Ribas vs. Namajunas,Payton Talbott,Cameron Saaiman,Red,Bantamweight,0,Men,KO/TKO,2,3.0,...,0.00,7.62,2.73,-0.60,0.08,-0.28,0.00,0.43,-0.2,-0.91
4,UFC Fight Night: Ribas vs. Namajunas,Billy Quarantillo,Youssef Zalal,Blue,Featherweight,0,Men,Submission,2,3.0,...,0.00,-5.08,4.48,3.84,0.07,-0.11,-0.22,0.01,-0.2,-1.04


In [122]:
df.columns

Index(['event_name', 'r_fighter', 'b_fighter', 'winner', 'weight_class',
       'is_title_bout', 'gender', 'method', 'finish_round', 'total_rounds',
       'time_sec', 'referee', 'r_kd', 'r_sig_str', 'r_sig_str_att',
       'r_sig_str_acc', 'r_str', 'r_str_att', 'r_str_acc', 'r_td', 'r_td_att',
       'r_td_acc', 'r_sub_att', 'r_rev', 'r_ctrl_sec', 'r_wins_total',
       'r_losses_total', 'r_age', 'r_height', 'r_weight', 'r_reach',
       'r_stance', 'r_SLpM_total', 'r_SApM_total', 'r_sig_str_acc_total',
       'r_td_acc_total', 'r_str_def_total', 'r_td_def_total', 'r_sub_avg',
       'r_td_avg', 'b_kd', 'b_sig_str', 'b_sig_str_att', 'b_sig_str_acc',
       'b_str', 'b_str_att', 'b_str_acc', 'b_td', 'b_td_att', 'b_td_acc',
       'b_sub_att', 'b_rev', 'b_ctrl_sec', 'b_wins_total', 'b_losses_total',
       'b_age', 'b_height', 'b_weight', 'b_reach', 'b_stance', 'b_SLpM_total',
       'b_SApM_total', 'b_sig_str_acc_total', 'b_td_acc_total',
       'b_str_def_total', 'b_td_def_total', 'b_

In [123]:
#Found null values after running the reverse order of three_rnd_df
df = df.dropna()

In [124]:
df[['finish_round', 'total_rounds', 'time_sec']]

,finish_round,total_rounds,time_sec
0,5,5.0,300
1,3,3.0,300
2,1,3.0,273
3,2,3.0,21
4,2,3.0,110
...,...,...,...
7184,3,3.0,300
7207,5,5.0,300
7250,1,1.0,720
7263,1,1.0,44


In [125]:
#Finding the target values
df['target'] = (df['finish_round'] -1)*300 + df['time_sec']

In [126]:
# Preparing df to be in order to prepare the train, valid, and test datasets
df = df.iloc[::-1]

In [131]:
df = df.reset_index()
df = df.drop(columns = ['level_0','index'], axis = 1)
df

,event_name,r_fighter,b_fighter,winner,weight_class,is_title_bout,gender,method,finish_round,total_rounds,...,reach_diff,SLpM_total_diff,SApM_total_diff,sig_str_acc_total_diff,td_acc_total_diff,str_def_total_diff,td_def_total_diff,sub_avg_diff,td_avg_diff,target
0,UFC 15: Collision Course,Randy Couture,Vitor Belfort,Red,Heavyweight,0,Men,KO/TKO,1,1.0,...,2.54,1.26,-1.30,0.11,-0.14,0.09,0.05,-0.1,1.76,496
1,UFC - Ultimate Brazil,Vitor Belfort,Wanderlei Silva,Red,Middleweight,0,Men,KO/TKO,1,1.0,...,0.00,-1.20,0.64,0.06,0.07,-0.10,-0.09,0.0,0.05,44
2,UFC 19: Ultimate Young Guns,Jeremy Horn,Chuck Liddell,Red,Middleweight,0,Men,Submission,1,1.0,...,-5.08,-1.78,-0.72,0.04,-0.37,-0.01,-0.52,1.1,1.39,720
3,UFC 25: Ultimate Japan 3,Tito Ortiz,Wanderlei Silva,Red,UFC Light Heavyweight Title,1,Men,Decision - Unanimous,5,5.0,...,0.00,-0.20,0.69,0.06,-0.18,0.00,-0.08,0.0,1.50,1500
4,UFC 29: Defense of the Belts,Chuck Liddell,Jeff Monson,Red,Middleweight,0,Men,Decision - Unanimous,3,3.0,...,5.08,2.00,-0.16,0.05,0.63,-0.03,0.15,-0.7,-0.62,900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6356,UFC Fight Night: Ribas vs. Namajunas,Billy Quarantillo,Youssef Zalal,Blue,Featherweight,0,Men,Submission,2,3.0,...,-5.08,4.48,3.84,0.07,-0.11,-0.22,0.01,-0.2,-1.04,410
6357,UFC Fight Night: Ribas vs. Namajunas,Payton Talbott,Cameron Saaiman,Red,Bantamweight,0,Men,KO/TKO,2,3.0,...,7.62,2.73,-0.60,0.08,-0.28,0.00,0.43,-0.2,-0.91,321
6358,UFC Fight Night: Ribas vs. Namajunas,Edmen Shahbazyan,AJ Dobson,Red,Middleweight,0,Men,KO/TKO,1,3.0,...,-2.54,-0.69,-1.22,0.06,-0.37,-0.01,-0.02,0.3,0.57,273
6359,UFC Fight Night: Ribas vs. Namajunas,Karl Williams,Justin Tafa,Red,Heavyweight,0,Men,Decision - Unanimous,3,3.0,...,12.70,-1.22,-3.32,-0.02,0.50,0.13,0.50,0.2,4.75,900
